In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
from pprint import pprint

In [3]:
import numpy as np
import decimal
import copy

In [4]:
import streamm

In this getting started example we will calculate the coupling between P3HT oligomers

Now let's create project and resource to keep track of our work

In [5]:
p3ht_et = streamm.Project('P3HT_ET')

And a resource object to keep track of where our files are 

In [6]:
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [7]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

In [8]:
res_local.make_dir()

In [9]:
p3ht_et.set_resource(res_local)

Now we need to set a remote resource we have ssh access to and run calculations 

In [10]:
peregrine = streamm.Resource('peregrine')

In [11]:
peregrine.meta['type'] = "ssh"
peregrine.ssh['username'] = "tkemper"    
peregrine.ssh['address'] = "peregrine.hpc.nrel.gov"    
peregrine.dir['storage'] = '/mss/users/%s'%(peregrine.ssh['username'])
peregrine.dir['scratch'] = '/scratch/%s'%(peregrine.ssh['username'])
peregrine.dir['home'] = res_local.dir['home']
peregrine.dir['launch'] = res_local.dir['launch']
peregrine.dir['templates'] = res_local.dir['templates']

# Set default simulation specs
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['e-mail'] = 'travis.kemper@nrel.gov'
peregrine.properties['scratch'] = peregrine.dir['scratch']
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['walltime'] = 4
peregrine.properties['nodes'] = int(1)
peregrine.properties['ppn'] = int(24)
peregrine.properties['nproc'] = peregrine.properties['nodes']*peregrine.properties['ppn']
peregrine.properties['queue'] = 'short'
peregrine.properties['feature'] = '24core'
peregrine.properties['exe_command'] = 'qsub '

In [12]:
pprint(peregrine.properties)

{u'allocation': 'orgopv',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': '/scratch/tkemper',
 u'walltime': 4}


In [13]:
peregrine.make_dir()

Explicitely create a thiophene molecule

In [14]:
bbTh = streamm.Buildingblock('thiophene')
symbols = ['C','C','C','C','S','H','H','H','H']
positions = [ ]
positions.append([-1.55498576,-1.91131218,-0.00081000])
positions.append([-0.17775976,-1.91131218,-0.00081000])
positions.append([0.34761524,-0.57904218,-0.00081000])
positions.append([-0.65884476,0.36101082,0.00000000])
positions.append([-2.16948076,-0.35614618,-0.00000800])
positions.append([-2.18966076,-2.79526518,-0.00132100])
positions.append([0.45389024,-2.80145418,-0.00106400])
positions.append([1.41682424,-0.35961818,-0.00138200])
positions.append([-0.51943676,1.44024682,0.00064700])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbTh.add_partpos(pt_i,pos_i)

In [15]:
bbTh.particles[5].rsite = 'termcap'
bbTh.particles[6].rsite = 'funccap'
bbTh.particles[8].rsite = 'termcap'

Set some properties of the molecule to keep track of the parts

In [16]:
c_cnt =1
h_cnt =1

for pkey_i, particle_i  in bbTh.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)

                particle_i.resname = "SCP2"
                particle_i.residue = 1
                particle_i.charge = -0.025

                c_cnt +=1 
            if( particle_i.symbol == 'S' ):
                particle_i.resname = "ThS"
                particle_i.residue = 2
                particle_i.charge = -0.3

            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)

                particle_i.resname = "HA"
                particle_i.residue = 3
                particle_i.charge = 0.1

                h_cnt +=1 
                

Set the force-field type and guess some reasonable charges 

In [17]:
for pkey_i, particle_i  in bbTh.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.paramkey = 'CA'
                particle_i.charge = -0.025
            if( particle_i.symbol == 'S' ):
                particle_i.paramkey = 'S'
                particle_i.charge = -0.3
            if( particle_i.symbol == 'H' ):
                particle_i.paramkey = 'HA'
                particle_i.charge = 0.1

Optimize structure with NWChem

But let's put it in a function this time

In [18]:
def nw_opt(project_i,bb_i,res_i):
    '''Optimize a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_i = streamm.NWChem('nw_opt_{}'.format(bb_i.tag))
    print nwchem_i.tag 
    # Add thiophene structure 
    nwchem_i.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_i.set_resource(res_i)
    # Make the local directories 
    nwchem_i.make_dir()
    #Change to the `launch` directory
    os.chdir(nwchem_i.dir['launch'])
    # Copy over templates
    nwchem_i.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_i.cp_file('templates','nw',"nwchem.nw",'templates','launch')
    # Read in templates files 
    nwchem_i.load_str('templates','nw')        
    nwchem_i.load_str('templates','run')
    # Set calculation properties 
    nwchem_i.properties['basis'] = '6-31g'
    nwchem_i.properties['method'] = 'UHF'
    nwchem_i.properties['charge'] = 0
    nwchem_i.properties['spin_mult'] = 1
    nwchem_i.properties['task'] = 'SCF optimize'
    nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()
    # 
    pprint(nwchem_i.properties)
    # Replace <key> with properties value 
    nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))
    nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
    nwchem_i.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_i.tag))
    #
    nwchem_i.add_file('output','log',"%s.log"%(nwchem_i.tag))
    # Save details in .json files 
    os.chdir(nwchem_i.dir['home'])
    p3ht_et.dump_json()
    # 
    os.chdir(nwchem_i.dir['launch'])
    # 
    nwchem_i.push()
    # 
    nwchem_i.run()
    # Add calculation to project
    project_i.add_calc(nwchem_i,deepcopy = True)
    # 
    return project_i 

In [19]:
p3ht_et = nw_opt(p3ht_et,bbTh,peregrine)

nw_opt_thiophene
{u'allocation': 'orgopv',
 'basis': '6-31g',
 'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.55498576      -1.91131218      -0.00081000 \n     C      -0.17775976      -1.91131218      -0.00081000 \n     C       0.34761524      -0.57904218      -0.00081000 \n     C      -0.65884476       0.36101082       0.00000000 \n     S      -2.16948076      -0.35614618      -0.00000800 \n     H      -2.18966076      -2.79526518      -0.00132100 \n     H       0.45389024      -2.80145418      -0.00106400 \n     H       1.41682424      -0.35961818      -0.00138200 \n     H      -0.51943676       1.44024682       0.00064700 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_opt_thiophene/',
 'spin_mult': 1,
 '

In [23]:
nwchem_i = p3ht_et.calculations['nw_opt_thiophene']

Check status unit finished

In [24]:
p3ht_et.check()

Calculation nw_opt_thiophene has status finished


In [25]:
nwchem_i.analysis()

Print energies 

In [26]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[-0.4732391, -0.3475018, -0.3290008, 0.1318747, 0.2130991, 0.2315597, 0.2416684, 0.269473, 0.2970013, 0.3198832]
-551.190758108


Check that the positions of the structure have been optimized 

In [27]:
for pk,p in bbTh.particles.iteritems():
    print pk,p.symbol,bbTh.positions[pk]

0 C [ -1.55498576e+00  -1.91131218e+00  -8.10000000e-04]
1 C [ -1.77759760e-01  -1.91131218e+00  -8.10000000e-04]
2 C [ 0.34761524 -0.57904218 -0.00081   ]
3 C [-0.65884476  0.36101082  0.        ]
4 S [ -2.16948076e+00  -3.56146180e-01  -8.00000000e-06]
5 H [ -2.18966076e+00  -2.79526518e+00  -1.32100000e-03]
6 H [  4.53890240e-01  -2.80145418e+00  -1.06400000e-03]
7 H [  1.41682424e+00  -3.59618180e-01  -1.38200000e-03]
8 H [ -5.19436760e-01   1.44024682e+00   6.47000000e-04]


In [30]:
print bbTh.positions

[[ -1.55498576e+00  -1.91131218e+00  -8.10000000e-04]
 [ -1.77759760e-01  -1.91131218e+00  -8.10000000e-04]
 [  3.47615240e-01  -5.79042180e-01  -8.10000000e-04]
 [ -6.58844760e-01   3.61010820e-01   0.00000000e+00]
 [ -2.16948076e+00  -3.56146180e-01  -8.00000000e-06]
 [ -2.18966076e+00  -2.79526518e+00  -1.32100000e-03]
 [  4.53890240e-01  -2.80145418e+00  -1.06400000e-03]
 [  1.41682424e+00  -3.59618180e-01  -1.38200000e-03]
 [ -5.19436760e-01   1.44024682e+00   6.47000000e-04]]


In [28]:
print nwchem_i.strucC.positions

[[ -1.52898837e+00  -1.96229356e+00  -8.69730000e-04]
 [ -1.87315280e-01  -1.91418741e+00  -9.73840000e-04]
 [  3.42636030e-01  -5.70352690e-01  -7.49680000e-04]
 [ -6.05136920e-01   3.80462320e-01   1.82600000e-05]
 [ -2.25051137e+00  -3.24275670e-01   2.70300000e-05]
 [ -2.16225171e+00  -2.82016718e+00  -1.23883000e-03]
 [  4.31102560e-01  -2.78813961e+00  -1.08293000e-03]
 [  1.39111433e+00  -3.53673110e-01  -1.27532000e-03]
 [ -4.82488110e-01   1.43973430e+00   5.87040000e-04]]


Update positions with optimized geometry 

In [32]:
for pk,p in bbTh.particles.iteritems():
    bbTh.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbTh.positions[pk]

0 C [ -1.52898837e+00  -1.96229356e+00  -8.69730000e-04]
1 C [ -1.87315280e-01  -1.91418741e+00  -9.73840000e-04]
2 C [ 0.34263603 -0.57035269 -0.00074968]
3 C [ -6.05136920e-01   3.80462320e-01   1.82600000e-05]
4 S [ -2.25051137e+00  -3.24275670e-01   2.70300000e-05]
5 H [ -2.16225171e+00  -2.82016718e+00  -1.23883000e-03]
6 H [  4.31102560e-01  -2.78813961e+00  -1.08293000e-03]
7 H [  1.39111433e+00  -3.53673110e-01  -1.27532000e-03]
8 H [ -4.82488110e-01   1.43973430e+00   5.87040000e-04]


Store the results in a tar ball in the storage directory 

In [29]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

Again let's make it a function

In [33]:
def nw_esp(project_i,bb_i,res_i):
    '''Calculate ESP charges of a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_esp = streamm.NWChem('nw_esp_{}'.format(bb_i.tag))
    print(nwchem_esp.tag)
    # Add thiophene structure with optimized coordinates from previous calculation
    nwchem_esp.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_esp.set_resource(res_i)
    # Add calculation to project
    project_i.add_calc(nwchem_esp)
    # Make the local directories 
    nwchem_esp.make_dir()
    # Change to the `launch` directory
    os.chdir(nwchem_esp.dir['launch'])
    #
    nwchem_esp.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_esp.cp_file('templates','nw',"nwchem.nw",'templates','launch')
    #
    nwchem_esp.load_str('templates','nw')        
    nwchem_esp.load_str('templates','run')
    # 
    nwchem_esp.properties['basis'] = '6-31g'
    nwchem_esp.properties['method'] = 'UHF'
    nwchem_esp.properties['charge'] = 0
    nwchem_esp.properties['spin_mult'] = 1
    nwchem_esp.properties['task'] = 'esp'
    nwchem_esp.properties['coord'] = nwchem_esp.strucC.write_coord()

    pprint(nwchem_esp.properties)

    nwchem_esp.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_esp.tag))

    nwchem_esp.properties['input_nw'] = nwchem_esp.files['input']['nw']
    nwchem_esp.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_esp.tag))

    nwchem_esp.add_file('output','log',"%s.log"%(nwchem_esp.tag))

    # Save details in .json files 

    os.chdir(nwchem_esp.dir['home'])
    project_i.dump_json()

    os.chdir(nwchem_esp.dir['launch'])
    nwchem_esp.push()
    nwchem_esp.run()
    # Add calculation to project
    project_i.add_calc(nwchem_esp,deepcopy = True)
    # 
    return project_i 
    
    

In [34]:
p3ht_et = nw_esp(p3ht_et,bbTh,peregrine)

nw_esp_thiophene
{u'allocation': 'orgopv',
 'basis': '6-31g',
 'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.52898837      -1.96229356      -0.00086973 \n     C      -0.18731528      -1.91418741      -0.00097384 \n     C       0.34263603      -0.57035269      -0.00074968 \n     C      -0.60513692       0.38046232       0.00001826 \n     S      -2.25051137      -0.32427567       0.00002703 \n     H      -2.16225171      -2.82016718      -0.00123883 \n     H       0.43110256      -2.78813961      -0.00108293 \n     H       1.39111433      -0.35367311      -0.00127532 \n     H      -0.48248811       1.43973430       0.00058704 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_esp_thiophene/',
 'spin_mult': 1,
 '

Check status unit finished

In [40]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_esp_thiophene has status running


In [36]:
nwchem_i = p3ht_et.calculations['nw_esp_thiophene']

In [37]:
nwchem_i.analysis()

In [62]:
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge

0 C -0.025
1 C -0.025
2 C -0.025
3 C -0.025
4 S -0.3
5 H 0.1
6 H 0.1
7 H 0.1
8 H 0.1


Print energies 

In [39]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[]
0.0


Update the charges of the Buildingblock

In [63]:
for pk,p in bbTh.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge
    print pk,p.symbol,p.charge

0 C -0.025
1 C -0.025
2 C -0.025
3 C -0.025
4 S -0.3
5 H 0.1
6 H 0.1
7 H 0.1
8 H 0.1


Store the results 

In [ ]:
nwchem_i.store()

Check that the positions of the structure have been optimized 

In [ ]:
for pk,p in nwchem_esp.strucC.particles.iteritems():
    print pk,p.symbol,p.charge

Store the results in a tar ball in the storage directory 

In [ ]:
nwchem_i.store()

In [ ]:
nwchem_i.pull()

In [41]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_esp_thiophene has status running


Build hexane

In [42]:
bbHex = streamm.Buildingblock('hexane')
symbols = ['C','H','H','H','C','H','H','C','H','H','C','H','H','C','H','H','C','H','H','H']
positions = [ ]
positions.append([-6.410969,-0.381641,-0.000031])
positions.append([-7.310084,0.245311,-0.000038])
positions.append([-6.456117,-1.028799,0.884636])
positions.append([-6.456111,-1.028812,-0.884689])
positions.append([-5.135268,0.467175,-0.000033])
positions.append([-5.135484,1.128782,0.877977])
positions.append([-5.135479,1.128771,-0.87805])
positions.append([-3.850566,-0.371258,-0.000024])
positions.append([-3.85112,-1.033978,0.87841])
positions.append([-3.851114,-1.033987,-0.878451])
positions.append([-2.567451,0.469603,-0.000024])
positions.append([-2.567784,1.132155,0.8784])
positions.append([-2.567776,1.132146,-0.878455])
positions.append([-1.283527,-0.370234,-0.000013])
positions.append([-1.28337,-1.032804,0.87836])
positions.append([-1.28336,-1.032812,-0.87838])
positions.append([0.00482234,0.47342231,-0.00000898])
positions.append([0.02595107,1.09220686,0.87266464])
positions.append([0.85585781,-0.17514133,0.00194589])
positions.append([0.02780957,1.08937798,-0.87463473])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbHex.add_partpos(pt_i,pos_i)

In [43]:
bbHex.particles[0].rsite = 'rg'
bbHex.particles[1].rsite = 'rgcap'

In [44]:
c_cnt =1
h_cnt =1
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)
                particle_i.resname = "SCP3"
                particle_i.residue = c_cnt
                c_cnt +=1 
            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)
                particle_i.resname = "HC"
                particle_i.residue = c_cnt -1 
                h_cnt +=1 

Set the parameter keys and some reasonable atomic charges 

In [45]:
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.paramkey = 'CT'
                particle_i.charge = -0.12

            if( particle_i.symbol == 'H' ):
                particle_i.paramkey = 'HC'
                particle_i.charge = 0.06
            print pkey_i, particle_i.symbol,particle_i.charge

0 C -0.12
1 H 0.06
2 H 0.06
3 H 0.06
4 C -0.12
5 H 0.06
6 H 0.06
7 C -0.12
8 H 0.06
9 H 0.06
10 C -0.12
11 H 0.06
12 H 0.06
13 C -0.12
14 H 0.06
15 H 0.06
16 C -0.12
17 H 0.06
18 H 0.06
19 H 0.06


In [46]:
bbHex.particles[0].charge  = -0.18
bbHex.particles[16].charge  = -0.18

In [47]:
total_charge = 0.0
for pkey_i, particle_i  in bbHex.particles.iteritems():
            total_charge += particle_i.charge 

Check that the molecule is neutral 

In [49]:
print total_charge

0.0


Now let us optimze and calculate ESP charges for hexane

Optimize structure with NWChem

In [50]:
p3ht_et = nw_opt(p3ht_et,bbHex)

nw_opt_hexane
{u'allocation': 'orgopv',
 'basis': '6-31g',
 'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -6.41096900      -0.38164100      -0.00003100 \n     H      -7.31008400       0.24531100      -0.00003800 \n     H      -6.45611700      -1.02879900       0.88463600 \n     H      -6.45611100      -1.02881200      -0.88468900 \n     C      -5.13526800       0.46717500      -0.00003300 \n     H      -5.13548400       1.12878200       0.87797700 \n     H      -5.13547900       1.12877100      -0.87805000 \n     C      -3.85056600      -0.37125800      -0.00002400 \n     H      -3.85112000      -1.03397800       0.87841000 \n     H      -3.85111400      -1.03398700      -0.87845100 \n     C      -2.56745100       0.46960300      -0.00002400 \n     H      -2.56778400       1.13215500       0.87840000 \n     H      -2.56777600       1.13214600      -0.87845500 \n     C      -1.28352700      -0.37023400      -0.0000130

In [53]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_opt_hexane has status finished
Calculation nw_esp_thiophene has status running


Get the calculation from the project object 

In [54]:
nw_hex_opt = p3ht_et.calculations['nw_opt_hexane']

In [55]:
nw_hex_opt.analysis()

Print energies 

In [56]:
print nw_hex_opt.properties['alpha_energies'][10:20]
print nw_hex_opt.properties['energy']

[-0.5549424, -0.5282407, -0.5218841, -0.4996812, -0.4774207, -0.4626707, -0.453462, -0.4506618, -0.4351213, 0.2272358]
-235.271544114


Check that the positions of the structure have been optimized 

In [57]:
for pk,p in bbHex.particles.iteritems():
    print pk,p.symbol,bbHex.positions[pk]

0 C [ -6.41096900e+00  -3.81641000e-01  -3.10000000e-05]
1 H [ -7.31008400e+00   2.45311000e-01  -3.80000000e-05]
2 H [-6.456117 -1.028799  0.884636]
3 H [-6.456111 -1.028812 -0.884689]
4 C [ -5.13526800e+00   4.67175000e-01  -3.30000000e-05]
5 H [-5.135484  1.128782  0.877977]
6 H [-5.135479  1.128771 -0.87805 ]
7 C [ -3.85056600e+00  -3.71258000e-01  -2.40000000e-05]
8 H [-3.85112  -1.033978  0.87841 ]
9 H [-3.851114 -1.033987 -0.878451]
10 C [ -2.56745100e+00   4.69603000e-01  -2.40000000e-05]
11 H [-2.567784  1.132155  0.8784  ]
12 H [-2.567776  1.132146 -0.878455]
13 C [ -1.28352700e+00  -3.70234000e-01  -1.30000000e-05]
14 H [-1.28337  -1.032804  0.87836 ]
15 H [-1.28336  -1.032812 -0.87838 ]
16 C [  4.82234000e-03   4.73422310e-01  -8.98000000e-06]
17 H [ 0.02595107  1.09220686  0.87266464]
18 H [ 0.85585781 -0.17514133  0.00194589]
19 H [ 0.02780957  1.08937798 -0.87463473]


In [58]:
print nw_hex_opt.strucC.positions

[[ -6.40791281e+00  -3.79034960e-01   1.35200000e-05]
 [ -7.29469957e+00   2.45353620e-01   4.91400000e-05]
 [ -6.45260791e+00  -1.01762866e+00   8.76427580e-01]
 [ -6.45264656e+00  -1.01758899e+00  -8.76423140e-01]
 [ -5.13217610e+00   4.67795080e-01   8.87000000e-06]
 [ -5.13142221e+00   1.11955396e+00   8.70240910e-01]
 [ -5.13154143e+00   1.11973388e+00  -8.70088870e-01]
 [ -3.85235227e+00  -3.76602540e-01  -1.56280000e-04]
 [ -3.85445249e+00  -1.02908886e+00   8.70797560e-01]
 [ -3.85461933e+00  -1.02901446e+00  -8.71171130e-01]
 [ -2.57064108e+00   4.64482030e-01  -2.78630000e-04]
 [ -2.56847864e+00   1.11712811e+00   8.70499540e-01]
 [ -2.56834855e+00   1.11675392e+00  -8.71321940e-01]
 [ -1.29138675e+00  -3.80706920e-01   1.11960000e-04]
 [ -1.29289220e+00  -1.03215144e+00   8.70561890e-01]
 [ -1.29267466e+00  -1.03267119e+00  -8.69963990e-01]
 [ -1.48075200e-02   4.65241570e-01   5.48200000e-05]
 [  2.98155000e-02   1.10488223e+00   8.75770120e-01]
 [  8.71480520e-01  -1.59796

Update positions in Buildingblock object

In [59]:
for pk,p in bbHex.particles.iteritems():
    bbHex.positions[pk] = nw_hex_opt.strucC.positions[pk]
    print pk,p.symbol,bbHex.positions[pk]

0 C [ -6.40791281e+00  -3.79034960e-01   1.35200000e-05]
1 H [ -7.29469957e+00   2.45353620e-01   4.91400000e-05]
2 H [-6.45260791 -1.01762866  0.87642758]
3 H [-6.45264656 -1.01758899 -0.87642314]
4 C [ -5.13217610e+00   4.67795080e-01   8.87000000e-06]
5 H [-5.13142221  1.11955396  0.87024091]
6 H [-5.13154143  1.11973388 -0.87008887]
7 C [ -3.85235227e+00  -3.76602540e-01  -1.56280000e-04]
8 H [-3.85445249 -1.02908886  0.87079756]
9 H [-3.85461933 -1.02901446 -0.87117113]
10 C [ -2.57064108e+00   4.64482030e-01  -2.78630000e-04]
11 H [-2.56847864  1.11712811  0.87049954]
12 H [-2.56834855  1.11675392 -0.87132194]
13 C [ -1.29138675e+00  -3.80706920e-01   1.11960000e-04]
14 H [-1.2928922  -1.03215144  0.87056189]
15 H [-1.29267466 -1.03267119 -0.86996399]
16 C [ -1.48075200e-02   4.65241570e-01   5.48200000e-05]
17 H [ 0.0298155   1.10488223  0.87577012]
18 H [ 0.87148052 -0.15979691  0.0015047 ]
19 H [ 0.03122486  1.10284434 -0.8770748 ]


Store the results in a tar ball in the storage directory 

In [60]:
nw_hex_opt.store()

Now let us calculate the ESP charges to use in our forcefield 

In [64]:
p3ht_et = nw_esp(p3ht_et,bbHex,peregrine)

nw_esp_hexane
{u'allocation': 'orgopv',
 'basis': '6-31g',
 'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -6.40791281      -0.37903496       0.00001352 \n     H      -7.29469957       0.24535362       0.00004914 \n     H      -6.45260791      -1.01762866       0.87642758 \n     H      -6.45264656      -1.01758899      -0.87642314 \n     C      -5.13217610       0.46779508       0.00000887 \n     H      -5.13142221       1.11955396       0.87024091 \n     H      -5.13154143       1.11973388      -0.87008887 \n     C      -3.85235227      -0.37660254      -0.00015628 \n     H      -3.85445249      -1.02908886       0.87079756 \n     H      -3.85461933      -1.02901446      -0.87117113 \n     C      -2.57064108       0.46448203      -0.00027863 \n     H      -2.56847864       1.11712811       0.87049954 \n     H      -2.56834855       1.11675392      -0.87132194 \n     C      -1.29138675      -0.38070692       0.0001119

Check status unit finished

In [65]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation nw_esp_thiophene has status running
Calculation nw_esp_hexane has status written


In [66]:
nwchem_i = p3ht_et.calculations['nw_esp_hexane']

In [67]:
nwchem_i.analysis()

In [68]:
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge

0 C -0.18
1 H 0.06
2 H 0.06
3 H 0.06
4 C -0.12
5 H 0.06
6 H 0.06
7 C -0.12
8 H 0.06
9 H 0.06
10 C -0.12
11 H 0.06
12 H 0.06
13 C -0.12
14 H 0.06
15 H 0.06
16 C -0.18
17 H 0.06
18 H 0.06
19 H 0.06


Print energies 

In [69]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[]
0.0


Update the charges of the Buildingblock

In [70]:
for pk,p in bbHex.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge
    print pk,p.symbol,p.charge

0 C -0.18
1 H 0.06
2 H 0.06
3 H 0.06
4 C -0.12
5 H 0.06
6 H 0.06
7 C -0.12
8 H 0.06
9 H 0.06
10 C -0.12
11 H 0.06
12 H 0.06
13 C -0.12
14 H 0.06
15 H 0.06
16 C -0.18
17 H 0.06
18 H 0.06
19 H 0.06


Store the results 

In [ ]:
nwchem_i.store()

Now we have optimized geometries and ESP Charges for each buildingblock

So let us make some P3HT oligomers 

First we need to identify the bonding within the Buildingblock

In [75]:
bbTh.bonded_nblist = bbTh.guess_nblist(0,radii_buffer=1.35)

In [80]:
bbTh.bonded_bonds()
bbTh.bonded_angles()
bbTh.bonded_dih()

In [76]:
bbHex.bonded_nblist = bbHex.guess_nblist(0,radii_buffer=1.35)

In [81]:
bbHex.bonded_bonds()
bbHex.bonded_angles()
bbHex.bonded_dih()

In [77]:
bbTh.find_rsites()
bbHex.find_rsites()

In [79]:
print(bbTh.show_rsites())

rsite:termcap[ paticle:atom[5] H1 (H) index:5 n_bonds:1] 
rsite:termcap[ paticle:atom[8] H4 (H) index:8 n_bonds:1] 
rsite:funccap[ paticle:atom[6] H2 (H) index:6 n_bonds:1] 



In [82]:
print(bbHex.show_rsites())

rsite:rg[ paticle:atom[0] C1 (C) index:0 n_bonds:4] 
rsite:rgcap[ paticle:atom[1] H1 (H) index:1 n_bonds:1] 



Sweet as bro!